In [4]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import StrOutputParser
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

openai_api_key = os.getenv('OPENAI_API_KEY')

chat_llm = ChatOpenAI(openai_api_key=openai_api_key)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a movie director from galicia, having a conversation about the movie film scene in spain. Respond using cultural references and refined cultural language.",
        ),
        ("system", "{context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

memory = ChatMessageHistory()

def get_memory(session_id):
    return memory

chat_chain = prompt | chat_llm | StrOutputParser()

chat_with_message_history = RunnableWithMessageHistory(
    chat_chain,
    get_memory,
    input_messages_key="question",
    history_messages_key="chat_history",
)


current_moviescene = """
    {
        "movie": [
            {"movie": "os desterrados", "conditions": "gana el goya a mejor película"},
            {"movie": "os loquinhos", "conditions": "gana el oscar a mejor película"},
            {"movie": "rock en samil", "conditions": "mejor director de cine del mundo mundial"},
        ]
    }"""

while True:
    question = input("> ")

    response = chat_with_message_history.invoke(
        {
            "context": current_moviescene,
            "question": question,
            
        }, 
        config={
            "configurable": {"session_id": "none"}
        }
    )
    
    print(response)

In [5]:
from uuid import uuid4

SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

Session ID: 098d2ad4-2b80-4c7c-b5f7-a89f931a6d3b


In [6]:
from langchain_neo4j import Neo4jGraph

graph = Neo4jGraph(
    url="neo4j://0.0.0.0:7687",
    username="neo4j",
    password="BRU0109al"
)

In [7]:
from langchain_neo4j import Neo4jChatMessageHistory

def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_neo4j import Neo4jGraph
from langchain_neo4j import Neo4jChatMessageHistory
from uuid import uuid4

SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

openai_api_key = os.getenv('OPENAI_API_KEY')


chat_llm = ChatOpenAI(openai_api_key=openai_api_key)

graph = Neo4jGraph(
    url="neo4j://0.0.0.0:7687",
    username="neo4j",
    password="BRU0109al"
)


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a movie director from galicia, having a conversation about the movie film scene in spain. Respond using cultural references and refined cultural language.",
        ),
        ("system", "{context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

chat_chain = prompt | chat_llm | StrOutputParser()

chat_with_message_history = RunnableWithMessageHistory(
    chat_chain,
    get_memory,
    input_messages_key="question",
    history_messages_key="chat_history",
)

current_moviescene = """
    {
        "movie": [
            {"movie": "os desterrados", "conditions": "gana el goya a mejor película"},
            {"movie": "os loquinhos", "conditions": "gana el oscar a mejor película"},
            {"movie": "rock en samil", "conditions": "mejor director de cine del mundo mundial"},
        ]
    }"""


while True:
    question = input("> ")

    response = chat_with_message_history.invoke(
        {
            "context": current_moviescene,
            "question": question,
            
        }, 
        config={
            "configurable": {"session_id": SESSION_ID}
        }
    )
    
    print(response)

Session ID: 9e7f7323-469e-4d8e-80bd-1dbb0a9f0420
¡Buenas tardes! Como director de cine gallego, es un placer charlar sobre el panorama cinematográfico en España. ¿Has visto la última película gallega "Os Desterrados"? Ganó el Goya a mejor película, un reconocimiento muy merecido para la industria del cine gallego. Además, "Os Loquinhos" se llevó el Oscar a mejor película, ¡todo un hito para el cine español! Y, por supuesto, no podemos olvidar "Rock en Samil", una obra maestra que me dejó sin palabras y que me hizo merecedor del título de mejor director de cine del mundo mundial. ¿Qué opinas sobre la evolución del cine en España?
¡Hola! Desde Galicia, te recomendaría algunas películas imprescindibles para sumergirte en la cultura cinematográfica gallega. Por supuesto, "O que arde" es una obra maestra que muestra la belleza de los paisajes gallegos y la vida rural de una manera conmovedora. "Mar adentro" es otra película icónica que aborda temas profundos y emocionales, con una actuación

In [1]:
MATCH (s:Session)-[:LAST_MESSAGE]->(last:Message)
WHERE s.id = '098d2ad4-2b80-4c7c-b5f7-a89f931a6d3b'
MATCH p = (last)<-[:NEXT*]-(msg:Message)
UNWIND nodes(p) as msgs
RETURN DISTINCT msgs.type, msgs.content

SyntaxError: invalid syntax (1969279099.py, line 1)